In [1]:
# Let's load the precompuded differential data (can take about 10mins)
include("../differentials_computation/sl4_utils.jl");

  Activating project at `~/Desktop/LowCohomologySOS`
[ Info: Precompiling LowCohomologySOS [70c9f613-c550-58ca-b626-6da403f9846d]


In [2]:
# Sanity checks for vanishing compositions of differentials

In [3]:
d₄x = LowCohomologySOS.embed.(identity, d₄, Ref(RG_Δ[4]));
d₅x = LowCohomologySOS.embed.(identity, d₅, Ref(RG_Δ[4]));
@info d₄x*d₅x == [zero(RG_Δ[4]) zero(RG_Δ[4]) zero(RG_Δ[4]) zero(RG_Δ[4])]
d₅x = LowCohomologySOS.embed.(identity, d₅, Ref(RG_Δ[5]));
d₆x = LowCohomologySOS.embed.(identity, d₆, Ref(RG_Δ[5]));
@info d₅x*d₆x == [
    zero(RG_Δ[5]) zero(RG_Δ[5]) zero(RG_Δ[5]) zero(RG_Δ[5]);
    zero(RG_Δ[5]) zero(RG_Δ[5]) zero(RG_Δ[5]) zero(RG_Δ[5]);
    zero(RG_Δ[5]) zero(RG_Δ[5]) zero(RG_Δ[5]) zero(RG_Δ[5])
]
d₆x = LowCohomologySOS.embed.(identity, d₆, Ref(RG_Δ[6]));
d₇x = LowCohomologySOS.embed.(identity, d₇, Ref(RG_Δ[6]));
@info d₆x*d₇x == [
    zero(RG_Δ[6]) zero(RG_Δ[6]);
    zero(RG_Δ[6]) zero(RG_Δ[6]);
    zero(RG_Δ[6]) zero(RG_Δ[6]);
    zero(RG_Δ[6]) zero(RG_Δ[6])
]
d₇x = LowCohomologySOS.embed.(identity, d₇, Ref(RG_Δ[7]));
d₈x = LowCohomologySOS.embed.(identity, d₈, Ref(RG_Δ[7]));
@info d₇x*d₈x == [
    zero(RG_Δ[7]) zero(RG_Δ[7]);
    zero(RG_Δ[7]) zero(RG_Δ[7]);
    zero(RG_Δ[7]) zero(RG_Δ[7]);
    zero(RG_Δ[7]) zero(RG_Δ[7])
]

[ Info: true
[ Info: true
[ Info: true
[ Info: true


# Computing Laplacians

In [4]:
# The stabiliser parts which we have to add to get free modules.
# Hopefully the stabilisers' elements belong to the half_bases.

dim3_stab_part = reshape([one(RG_Δ[3])-averaged_rep(m3_stabs[1], half_basis_Δ[3], RG_Δ[3])], 1, 1)
dim4_stab_part = [
    one(RG_Δ[4])-averaged_rep(m4_stabs[1], half_basis_Δ[4], RG_Δ[4]) zero(RG_Δ[4]) zero(RG_Δ[4]);
    zero(RG_Δ[4]) one(RG_Δ[4])-averaged_rep(m4_stabs[2], half_basis_Δ[4], RG_Δ[4]) zero(RG_Δ[4]);
    zero(RG_Δ[4]) zero(RG_Δ[4]) one(RG_Δ[4])-averaged_rep(m4_stabs[3], half_basis_Δ[4], RG_Δ[4]);
]
dim5_stab_part = [
    one(RG_Δ[5])-averaged_rep(m5_stabs[1], half_basis_Δ[5], RG_Δ[5]) zero(RG_Δ[5]) zero(RG_Δ[5]) zero(RG_Δ[5]);
    zero(RG_Δ[5]) one(RG_Δ[5])-averaged_rep(m5_stabs[2], half_basis_Δ[5], RG_Δ[5]) zero(RG_Δ[5]) zero(RG_Δ[5]);
    zero(RG_Δ[5]) zero(RG_Δ[5]) one(RG_Δ[5])-averaged_rep(m5_stabs[3], half_basis_Δ[5], RG_Δ[5]) zero(RG_Δ[5]);
    zero(RG_Δ[5]) zero(RG_Δ[5]) zero(RG_Δ[5]) one(RG_Δ[5])-averaged_rep(m5_stabs[4], half_basis_Δ[5], RG_Δ[5])
]
dim6_stab_part = [
    one(RG_Δ[6])-averaged_rep(m6_stabs[1], half_basis_Δ[6], RG_Δ[6]) zero(RG_Δ[6]) zero(RG_Δ[6]) zero(RG_Δ[6]);
    zero(RG_Δ[6]) one(RG_Δ[6])-averaged_rep(m6_stabs[2], half_basis_Δ[6], RG_Δ[6]) zero(RG_Δ[6]) zero(RG_Δ[6]);
    zero(RG_Δ[6]) zero(RG_Δ[6]) one(RG_Δ[6])-averaged_rep(m6_stabs[3], half_basis_Δ[6], RG_Δ[6]) zero(RG_Δ[6]);
    zero(RG_Δ[6]) zero(RG_Δ[6]) zero(RG_Δ[6]) one(RG_Δ[6])-averaged_rep(m6_stabs[4], half_basis_Δ[6], RG_Δ[6])
]
dim7_stab_part = [
    one(RG_Δ[7])-averaged_rep(m7_stabs[1], half_basis_Δ[7], RG_Δ[7]) zero(RG_Δ[7]);
    zero(RG_Δ[7]) one(RG_Δ[7])-averaged_rep(m7_stabs[2], half_basis_Δ[7], RG_Δ[7])
];

In [5]:
# Compute the Laplacians (can take about 30mins).
# At the end, we embed the Laplacian into RG_star, the group ring
# with the same basis as RG but with twisted multiplciation, i.e.
# (1+g)(1+h)=1+g+h+g^(-1)h. This is needed to translate hermitian squares
# to the standard ones for the definition of the semi-definite optimization problem
# (solvers prefer standard squares to hermitian :).
# This has no effect on the shape of the Laplacian since we just embed it.
RG_Δ₃_star = LowCohomologySOS.group_ring(sln, half_basis_Δ[3], star_multiplication = true)
RG_Δ₄_star = LowCohomologySOS.group_ring(sln, half_basis_Δ[4], star_multiplication = true)
RG_Δ₅_star = LowCohomologySOS.group_ring(sln, half_basis_Δ[5], star_multiplication = true)
RG_Δ₆_star = LowCohomologySOS.group_ring(sln, half_basis_Δ[6], star_multiplication = true)
RG_Δ₇_star = LowCohomologySOS.group_ring(sln, half_basis_Δ[7], star_multiplication = true)

d₄x = LowCohomologySOS.embed.(identity, d₄, Ref(RG_Δ[3]));
Δ₃x = d₄x*d₄x'+dim3_stab_part

d₄x = LowCohomologySOS.embed.(identity, d₄, Ref(RG_Δ[4]));
d₅x = LowCohomologySOS.embed.(identity, d₅, Ref(RG_Δ[4]));
Δ₄x = d₄x'*d₄x+d₅x*d₅x'+dim4_stab_part

d₅x = LowCohomologySOS.embed.(identity, d₅, Ref(RG_Δ[5]));
d₆x = LowCohomologySOS.embed.(identity, d₆, Ref(RG_Δ[5]));
Δ₅x = d₅x'*d₅x+d₆x*d₆x'+dim5_stab_part

d₆x = LowCohomologySOS.embed.(identity, d₆, Ref(RG_Δ[6]));
d₇x = LowCohomologySOS.embed.(identity, d₇, Ref(RG_Δ[6]));
Δ₆x = d₆x'*d₆x+d₇x*d₇x'+dim6_stab_part

d₇x = LowCohomologySOS.embed.(identity, d₇, Ref(RG_Δ[7]));
d₈x = LowCohomologySOS.embed.(identity, d₈, Ref(RG_Δ[7]));
Δ₇x = d₇x'*d₇x+d₈x*d₈x'+dim7_stab_part

Δ₃ = LowCohomologySOS.embed.(identity, Δ₃x, Ref(RG_Δ₃_star))
Δ₄ = LowCohomologySOS.embed.(identity, Δ₄x, Ref(RG_Δ₄_star))
Δ₅ = LowCohomologySOS.embed.(identity, Δ₅x, Ref(RG_Δ₅_star))
Δ₆ = LowCohomologySOS.embed.(identity, Δ₆x, Ref(RG_Δ₆_star))
Δ₇ = LowCohomologySOS.embed.(identity, Δ₇x, Ref(RG_Δ₇_star))

2×2 Matrix{StarAlgebras.AlgebraElement{StarAlgebra{Groups.MatrixGroups.SpecialLinearGroup{4, Int8, DataType, Alphabet{Groups.MatrixGroups.ElementaryMatrix{4, Int8}}, Vector{Groups.MatrixGroups.ElementaryMatrix{4, Int8}}}, FPGroupElement{Groups.MatrixGroups.SpecialLinearGroup{4, Int8, DataType, Alphabet{Groups.MatrixGroups.ElementaryMatrix{4, Int8}}, Vector{Groups.MatrixGroups.ElementaryMatrix{4, Int8}}}, …}, …}, Rational{Int64}, SparseArrays.SparseVector{Rational{Int64}, Int64}}}:
 27//8·(id) -1//8·E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₄₁^-1*E₃₂^-1*E₃₂*E₂₃^-2*E₃₂*E₂₃^-2*E₄₂*E₄₃^-1*E₂₄^-1*E₁₄*E₂₃*E₁₃^-1*E₁₂^-1 -1//8·E₄₁*E₃₂^2*E₂₃^-2*E₃₂*E₂₃^-2*E₄₂^-1*E₄₃^-1*E₄₃*E₃₄^-2*E₄₃*E₃₄^-2*E₂₄^-1*E₁₄*E₂₃*E₁₃^-1*E₁₂^-1 -1//1·E₁₂^-1*E₂₁*E₁₂^-1*E₂₃^-1*E₃₂*E₂₃^-1*E₄₂*E₄₃^2*E₃₄^-2*E₄₃*E₃₄^-3*E₁₃^-1 -1//1·E₁₂^-1*E₂₁*E₁₂^-1*E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₂₃^-1*E₃₂*E₂₃^-1*E₃₂*E₂₃^-2*E₃₂*E₂₃^-2*E₄₂^-1*E₄₃*E₃₄^-1*E₁₃^-1 -1//16·E₂₁^2*E₁₂^-2*E₂₁*E₁₂^-2*E₃₁^-1*E₄₁^-2*E₃₂^-1*E₄₂^-1*E₄₃^2*E₃₄^-2*E₄₃*E₃₄^-3*E₁₃^-1*E₁₂^-1 -1//16·E₂₁*E₃₁

In [ ]:
# Check if the Laplacians are hermitian (just to be sure we have not spolied something obvious)
@info Δ₃' == Δ₃
@info Δ₄' == Δ₄
@info Δ₅' == Δ₅
@info Δ₆' == Δ₆
@info Δ₇' == Δ₇

In [6]:
# Load the solver ...
include("/home/mizerka/Desktop/LowCohomologySOS/scripts/optimizers.jl");

[ Info: Precompiling SCS [c946c3f1-0d1f-5ce8-9dea-7daa1f7e2d13]


In [7]:
# ... and the optimization package
using JuMP

# Finding spectral gaps

We want to find a numerical approximation: λ and Q such that Δₙ-λI ≈ 𝕩*QᵀQ𝕩,
where 𝕩 is the vector formed from half_basis as in https://arxiv.org/abs/2207.02783, Lemma 3.3.

## Δ₃

In [ ]:
I = reshape([one(RG_Δ₃_star)],1,1)
sos_problem = LowCohomologySOS.sos_problem(Δ₃, I, 0.05) # on the cost of optimality, bound the gap frome above for a quicker solution
JuMP.set_optimizer(sos_problem, scs_opt(eps = 1e-9, max_iters = 20_000))
JuMP.optimize!(sos_problem)

In [ ]:
# Certify the solution.
# For the certification theoretical background, see https://arxiv.org/abs/2207.02783, Proposition 3.2.
λ, Q = LowCohomologySOS.get_solution(sos_problem)
LowCohomologySOS.certify_sos_decomposition(Δ₃, I, λ, Q, half_basis_Δ[3])

## Δ₄

In [ ]:
I = [
    one(RG_Δ₄_star) zero(RG_Δ₄_star) zero(RG_Δ₄_star);
    zero(RG_Δ₄_star) one(RG_Δ₄_star) zero(RG_Δ₄_star);
    zero(RG_Δ₄_star) zero(RG_Δ₄_star) one(RG_Δ₄_star)
]
sos_problem = LowCohomologySOS.sos_problem(Δ₄, I, 0.05)
JuMP.set_optimizer(sos_problem, scs_opt(eps = 1e-9, max_iters = 20_000))
JuMP.optimize!(sos_problem)

In [ ]:
λ, Q = LowCohomologySOS.get_solution(sos_problem)
LowCohomologySOS.certify_sos_decomposition(Δ₄, I, λ, Q, half_basis_Δ[4])

## Δ₅

In [ ]:
I = [
    one(RG_Δ₅_star) zero(RG_Δ₅_star) zero(RG_Δ₅_star) zero(RG_Δ₅_star);
    zero(RG_Δ₅_star) one(RG_Δ₅_star) zero(RG_Δ₅_star) zero(RG_Δ₅_star);
    zero(RG_Δ₅_star) zero(RG_Δ₅_star) one(RG_Δ₅_star) zero(RG_Δ₅_star);
    zero(RG_Δ₅_star) zero(RG_Δ₅_star) zero(RG_Δ₅_star) one(RG_Δ₅_star)
]
sos_problem = LowCohomologySOS.sos_problem(Δ₅, I, 0.05)
JuMP.set_optimizer(sos_problem, scs_opt(eps = 1e-9, max_iters = 20_000))
JuMP.optimize!(sos_problem)

In [ ]:
λ, Q = LowCohomologySOS.get_solution(sos_problem)
LowCohomologySOS.certify_sos_decomposition(Δ₅, I, λ, Q, half_basis_Δ[5])

## Δ₆

In [ ]:
I = [
    one(RG_Δ₆_star) zero(RG_Δ₆_star) zero(RG_Δ₆_star) zero(RG_Δ₆_star);
    zero(RG_Δ₆_star) one(RG_Δ₆_star) zero(RG_Δ₆_star) zero(RG_Δ₆_star);
    zero(RG_Δ₆_star) zero(RG_Δ₆_star) one(RG_Δ₆_star) zero(RG_Δ₆_star);
    zero(RG_Δ₆_star) zero(RG_Δ₆_star) zero(RG_Δ₆_star) one(RG_Δ₆_star)
]
sos_problem = LowCohomologySOS.sos_problem(Δ₆, I, 0.05)
JuMP.set_optimizer(sos_problem, scs_opt(eps = 1e-9, max_iters = 20_000))
JuMP.optimize!(sos_problem)

In [ ]:
λ, Q = LowCohomologySOS.get_solution(sos_problem)
LowCohomologySOS.certify_sos_decomposition(Δ₆, I, λ, Q, half_basis_Δ[6])

## Δ₇

In [ ]:
I = [
    one(RG_Δ₇_star) zero(RG_Δ₇_star);
    zero(RG_Δ₇_star) one(RG_Δ₇_star)
]
sos_problem = LowCohomologySOS.sos_problem(Δ₇, I, 0.05)
JuMP.set_optimizer(sos_problem, scs_opt(eps = 1e-9, max_iters = 20_000))
JuMP.optimize!(sos_problem)

In [ ]:
λ, Q = LowCohomologySOS.get_solution(sos_problem)
LowCohomologySOS.certify_sos_decomposition(Δ₇, I, λ, Q, half_basis_Δ[7])